## API Tiingo - Diario
###### Helga Zambrana - Abril 2024

#### Importar las librerías necesarias

In [26]:
import requests
from decouple import config
import pandas as pd
import psycopg2
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float, DateTime
from sqlalchemy.dialects.postgresql import insert
import datetime

#### Leer el usuario, la contraseña y el token desde la variable de entorno

In [21]:
my_host = config('DATABASE_HOST')
port = config('DATABASE_PORT')
password = config('DATABASE_PASSWORD')
database_name = config('DATABASE_NAME')
user = config('DATABASE_USER')
token = config('TIINGO_TOKEN')

#### Definir la clase TiingoData para manejar los datos de Tiingo

In [22]:
# Definir la clase ListaSimbolos para manejar los símbolos
class ListaSimbolos:
    def __init__(self, simbolos):
        self.simbolos = simbolos
    
    def __call__(self):
        return self.simbolos

class TiingoData:
    # Inicializar atributos de la clase
    def __init__(self, token):
        self.token = token

    def get_data(self, lista_simbolos):
        simbolos = lista_simbolos()
        precios = []

        for simbolo in simbolos:
            url = f"https://api.tiingo.com/tiingo/daily/{simbolo}/prices"
            parametros = {
                'startDate': '2023-01-01',
                'endDate': datetime.date.today().isoformat(),
                'format': 'json',
            }
            headers = {
                'Authorization': f'Token {self.token}'  # Agregar el token enmascarado en los headers
            }

            try:
                respuesta = requests.get(url, params=parametros, headers=headers)
                respuesta.raise_for_status()  # Lanzar una excepción si la solicitud no es exitosa
                datos = respuesta.json()
                
                # Verificar la estructura de los datos
                if isinstance(datos, list) and all(isinstance(item, dict) for item in datos):
                    for dato in datos:
                        dato['ticker'] = simbolo  # Agregar el símbolo a cada fila porque si no, no lo trae
                    precios.extend(datos)
                else:
                    print(f"Los datos recibidos para el símbolo {simbolo} no están en el formato esperado.")
            except requests.exceptions.RequestException as e:
                print(f"Error al hacer la solicitud para el símbolo {simbolo}:", e)

        if precios:
            # Convertir los datos en un DataFrame de pandas
            df = pd.DataFrame(precios)
            # Filtrar y renombrar las columnas necesarias
            columnas = {
                'date': 'fecha',
                'ticker': 'simbolo',
                'open': 'apertura',
                'high': 'maximo',
                'low': 'minimo',
                'close': 'cierre',
                'volume': 'volumen',
            }
            df.rename(columns=columnas, inplace=True)
            # Convertir la columna de fecha a tipo datetime
            df['fecha'] = pd.to_datetime(df['fecha'])
            return df
        else:
            return None

#### Conectarse con Amazon Redshift y crear tabla (si no existe)

In [23]:
class BaseDatosRedshift:
    def __init__(self, host, port, database_name, user, password):
        self.host = host
        self.port = port
        self.database_name = database_name
        self.user = user
        self.password = password
        self.engine = None
        self.metadata = MetaData()

    def conectar(self):
        try:
            self.engine = create_engine(f'postgresql+psycopg2://{self.user}:{self.password}@{self.host}:{self.port}/{self.database_name}')
            print("Conexión a la base de datos exitosa.")
        except Exception as e:
            print(f"Error al crear la conexión a la base de datos:", e)

    def crear_tabla_tiingo(self):
        if self.engine is not None:
            try:
                tiingo_table = Table('tiingo', self.metadata,
                                     autoload_with=self.engine,
                                     schema='hvzambrana_coderhouse')
                tiingo_table.create(self.engine, checkfirst=True)
                print("Tabla ya existe.")
            except Exception as e:
                print(f"Error al crear la tabla:", e)
        else:
            print("No hay conexión a la base de datos.")

    def cargar_datos(self, df):
        if self.engine is not None:
            try:
                with self.engine.connect() as conn:
                    # Cargar datos usando la clave primaria compuesta para reemplazar filas duplicadas
                    df.to_sql('tiingo', con=conn, schema='hvzambrana_coderhouse', if_exists='replace', index=False, method='multi', chunksize=1000)
                print("Datos cargados exitosamente.")
            except Exception as e:
                print(f"Error al cargar los datos en la tabla:", e)
        else:
            print("No hay conexión a la base de datos.")

#### Obtener datos de Tiingo y cargarlos en Redshift

In [25]:
# Lista de símbolos a consultar
simbolos = ['AMZN', 'AAPL', 'DIS', 'GOOGL', 'JNJ', 'MCD', 'MELI', 'MSFT', 'NVDA', 'TSLA']
lista_simbolos = ListaSimbolos(simbolos)

# Crear instancia de TiingoData
tiingo_data = TiingoData(token)

# Obtener y mostrar los datos de Tiingo
datos_tiingo_df = tiingo_data.get_data(lista_simbolos)

# Crear instancia de la clase BaseDatosRedshift con los datos de conexión
bd_redshift = BaseDatosRedshift(my_host, port, database_name, user, password)

# Conectar con la base de datos
bd_redshift.conectar()

# Crear tabla en la base de datos
bd_redshift.crear_tabla_tiingo()

# Cargar los datos en Redshift
if datos_tiingo_df is not None:
    bd_redshift.cargar_datos(datos_tiingo_df)

Conexión a la base de datos exitosa.
Error al crear la tabla: (psycopg2.errors.UndefinedObject) unrecognized configuration parameter "standard_conforming_strings"

[SQL: show standard_conforming_strings]
(Background on this error at: https://sqlalche.me/e/20/f405)
Error al cargar los datos en la tabla: (psycopg2.errors.UndefinedObject) unrecognized configuration parameter "standard_conforming_strings"

[SQL: show standard_conforming_strings]
(Background on this error at: https://sqlalche.me/e/20/f405)
